In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# print("Key:", os.getenv("AZURE_OPENAI_API_KEY"))
# print("Endpoint:", os.getenv("azure_endpoint"))
# print('deployment:', os.getenv("azure_embedding_deployment"))


# create embeddings

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_deployment = "text-embedding-3-small",
    azure_endpoint = "https://eastus.api.cognitive.microsoft.com/",
    api_key = "27a3e6d835d44c298e146a73c7abdedd",

)

In [ ]:
def generate_embeddings(x):
    # generate the embedding for the given text and return embeddings
    return embeddings.embed_query(x)

In [ ]:
# read the data from source
input_df = pd.read_csv(r"/home/zadmin/Desktop/aksh_assignment/assignment2/assignment2dataset.csv")
input_df.head()

In [ ]:
input_df['title_desc'] = input_df.apply(lambda x: str({'title': x['title'], 'description': x['description']}), axis=1)


In [ ]:
input_df.head()

In [ ]:
# apply the embedding generation function on the title_desc column
input_df['title_desc_emb'] = input_df.apply(lambda x: generate_embeddings(x['title_desc']), axis=1)

# load vector db

In [ ]:
import chromadb
chroma_client = chromadb.Client()
collection_name = 'course_description'

In [ ]:
collection = chroma_client.create_collection(name=collection_name)

In [ ]:
input_df['course_id'].to_list()

In [ ]:
import uuid

ids = [str(uuid.uuid4()) for _ in range(len(input_df))]

In [ ]:
collection.add( 
    ids = ids,
    documents = input_df['title_desc'].to_list(),
    metadatas=  input_df.drop(columns=["title_desc", "title_desc_emb"], axis=1).to_dict(orient="records"), 
    embeddings = input_df['title_desc_emb'].to_list()   
)

In [ ]:
print("Number of docs dumped into vector DB")
print(len(collection.get()['ids']))

In [ ]:
# using vector db object to initialize a retriever object - to perform vector search/retriev

question = 'What is course I should take after "Internet of Things (IoT)", I am interested in Machine Learning and AI'
collection.query(query_embeddings = generate_embeddings(question),
                 n_results=2, 
                  )
